# Sentiment_Clf - Logistic Regression

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.pipeline import Pipeline
import joblib

# Load Data

In [2]:
df = pd.read_parquet(r"input_data\ccd_full.parquet")

print(df.shape)
display(df.head())                    

(25829, 3)


text  label label_name
0                      i am still waiting on my card      1   negative
1  what can i do if my card still hasnt arrived a...      1   negative
2  i have been waiting over a week is the card st...      1   negative
3  can i track my card while it is in the process...      2    neutral
4  how do i know if i will get my card or if it i...      2    neutral

In [3]:
# testing - check for duplicates or errors in labels
sorted(df['label_name'].unique())

['negative', 'neutral', 'positive', 'very negative', 'very positive']

In [4]:
df['label_name'].value_counts()

label_name
neutral          8105
negative         7455
positive         4363
very positive    3190
very negative    2716
Name: count, dtype: int64

In [5]:
lbl2id = {
    'very negative': 0,
    'negative': 1,
    'neutral': 2, 
    'positive': 3,
    'very positive': 4
}
id2lb = {
    0: 'very negative',
    1: 'negative',
    2: 'neutral', 
    3: 'positive',
    4: 'very positive'
}

In [6]:
train, test = train_test_split(df, test_size=0.3, stratify=df['label_name'], random_state=42)
test, validate = train_test_split(test, test_size=1/3, stratify=test['label_name'], random_state=42)

train.reset_index(drop=True, inplace=True)
validate.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)

train.shape, validate.shape, test.shape

((18080, 3), (2583, 3), (5166, 3))

In [7]:
train.head()

text  label label_name
0   i bought an item and the exchange rate was wrong      1   negative
1  i dont recognize the direct debit payment from...      1   negative
2     what is the expected delivery date for my card      2    neutral
3  please check the balance of my checking accoun...      2    neutral
4  photographed with melancholy richness and eloq...      2    neutral

In [8]:
X_train = train.loc[:, 'text']
y_train = train.loc[:, 'label_name']

X_validate = validate.loc[:, 'text']
y_validate = validate.loc[:, 'label_name']

X_test = test.loc[:, 'text']
y_test = test.loc[:, 'label_name']

In [9]:
X_train.shape, y_train.shape

((18080,), (18080,))

In [10]:
X_validate.shape, y_validate.shape

((2583,), (2583,))

In [11]:
X_test.shape, y_test.shape

((5166,), (5166,))

In [12]:
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),  # Feature extraction using TF-IDF
    ('classifier', LogisticRegression())  
])
pipeline.fit(X_train, y_train)

C:\py312_env_IssPlp\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('classifier', LogisticRegression())])

In [13]:
# Make predictions and evaluate the model
y_pred = pipeline.predict(X_validate)

# Print evaluation metrics
print("Accuracy:", round(accuracy_score(y_validate, y_pred), 3))
print(classification_report(y_validate, y_pred))

Accuracy: 0.587
               precision    recall  f1-score   support

     negative       0.58      0.69      0.63       745
      neutral       0.70      0.70      0.70       811
     positive       0.42      0.47      0.44       436
very negative       0.51      0.29      0.37       272
very positive       0.60      0.47      0.53       319

     accuracy                           0.59      2583
    macro avg       0.56      0.52      0.53      2583
 weighted avg       0.59      0.59      0.58      2583



##### Accuracy is lower than even tinyBERT

# Try MNB()

In [14]:
from sklearn.naive_bayes import MultinomialNB

pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),  # Feature extraction using TF-IDF
    ('classifier', MultinomialNB())  
])
pipeline.fit(X_train, y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('classifier', MultinomialNB())])

In [15]:
# Make predictions and evaluate the model
y_pred = pipeline.predict(X_validate)

# Print evaluation metrics
print("Accuracy:", round(accuracy_score(y_validate, y_pred), 3))
print(classification_report(y_validate, y_pred))

Accuracy: 0.551
               precision    recall  f1-score   support

     negative       0.51      0.82      0.63       745
      neutral       0.63      0.72      0.67       811
     positive       0.44      0.34      0.38       436
very negative       0.67      0.01      0.01       272
very positive       0.68      0.25      0.36       319

     accuracy                           0.55      2583
    macro avg       0.58      0.43      0.41      2583
 weighted avg       0.57      0.55      0.50      2583

